In [1]:
# !pip install dotenv
# !pip install paramiko
# !pip install fabric


In [2]:
import os
from dotenv import load_dotenv
import subprocess
import paramiko


load_dotenv()

True

In [3]:
# Load secrets from .env
SECRET_KEY = os.getenv("SECRET_KEY")
IMMICH_API_KEY = os.getenv("IMMICH_API_KEY")
IMMICH_UPLOAD_URL = str(os.getenv("IMMICH_UPLOAD_URL"))
SERVER_ADDRESS = str(os.getenv("SERVER_ADDRESS"))
SERVER_USERNAME = str(os.getenv("SERVER_USERNAME"))
SERVER_PASSWORD = str(os.getenv("SERVER_PASSWORD"))
# SSH_KEY_LOC = os.getenv("SSH_KEY_LOC")

TMP_DIR = "./tmp"

In [4]:
from fabric import Connection

# Create a connection using password authentication
conn = Connection(
    host="192.168.0.61",
    user="jstephencorey",
    connect_kwargs={
        "password": SERVER_PASSWORD
    }
)

# Verify it works
result = conn.run("hostname", hide=True)
print("Connected to:", result.stdout.strip())


Connected to: jserver


In [12]:
remote_root = f"/zstorage/surveillance_video"

video_name = "20251023_151848_clip_03785_1.mp4"

remote_filepath = f"{remote_root}/{video_name}"
local_filepath = f"{TMP_DIR}/{video_name}"

# Download (copy)
conn.get(remote_filepath, local_filepath)

# Delete the original (move)
conn.run(f"rm {remote_filepath}")

print("File moved successfully!")


File moved successfully!


In [13]:
import subprocess

output_file = video_name.split(".")[0] + ".mkv"
output_filepath = f"{TMP_DIR}/{output_file}"
assert not os.path.exists(output_filepath)
HIGH_QUALITY = "30"
LOW_QUALITY = "37" #Found through testing to be a good balance of size and not losing fidelity

cmd = [
    "ffmpeg",
    "-hwaccel", "cuda",
    "-i", local_filepath,
    "-c:v", "av1_nvenc",
    "-preset", 'p7',
    "-cq", LOW_QUALITY,
    "-c:a", "copy",
    output_filepath
]

subprocess.run(cmd, check=True)

CompletedProcess(args=['ffmpeg', '-hwaccel', 'cuda', '-i', './tmp/20251023_151848_clip_03785_1.mp4', '-c:v', 'av1_nvenc', '-preset', 'p7', '-cq', '37', '-c:a', 'copy', './tmp/20251023_151848_clip_03785_1.mkv'], returncode=0)